In [0]:
# import libraries necessary for this project
import os, sys, pickle

import numpy as np
import pandas as pd

from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression, SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve, accuracy_score
from sklearn.preprocessing import MinMaxScaler

# display for this notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [0]:
# 碼表物件

import time
import math

class StopWatch:
    def __init__(self):
        self.start()
    def start(self):
        self._startTime = time.time()
        self._stopTime = None
    def stop(self):
        self._stopTime = time.time()
    def getStartTime(self):
        return self._startTime
    def elapsed(self):
        if self._stopTime is None:
            self.stop()
        diff= self._stopTime - self._startTime
        hours, rem = divmod(diff, 3600)
        minutes, seconds = divmod(rem, 60)
        return "{:0>2}:{:0>2}:{:06.3f}".format(int(hours),int(minutes),seconds)
    
sw = StopWatch()    

In [5]:
if not os.path.isfile("train_offline.csv"):
  #shell command - 解壓縮
  !unzip ml100marathon-02-01

Archive:  ml100marathon-02-01.zip
  inflating: column_description.csv  
  inflating: train_offline.csv       
  inflating: sample_submission.csv   
  inflating: test_offline.csv        


In [0]:
dfoff = pd.read_csv('train_offline.csv')
dftest = pd.read_csv('test_offline.csv')

In [7]:
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN


In [8]:
print(dfoff.shape)

(1160742, 7)


In [11]:
print('Discount_rate 類型：\n',dfoff['Discount_rate'].unique())

Discount_rate 類型：
 [nan '20:1' '200:20' '10:5' '100:10' '200:30' '20:5' '30:5' '50:10'
 '150:10' '100:30' '200:50' '100:50' '300:30' '50:20' '0.9' '50:5'
 '150:20' '10:1' '30:10' '30:1' '0.95' '100:5' '50:1' '100:20' '0.8'
 '300:20' '100:1' '20:10' '0.85' '0.6' '5:1' '150:30' '200:10' '300:50'
 '0.5' '200:5' '0.7' '0.75' '30:20' '0.2' '150:50' '300:10' '50:30'
 '150:5' '200:100']


In [21]:
#print('Coupon_id 類型：\n',dfoff['Coupon_id'].unique())
#print('Coupon_id 數量：\n',dfoff['Coupon_id'].value_counts())

# 取得前20名熱門 coupon
top20_coupon = list(dfoff['Coupon_id'].value_counts())[0:21]
top20_coupon

[46676,
 26035,
 26009,
 24815,
 24500,
 21760,
 21693,
 21378,
 21059,
 16824,
 16058,
 13850,
 13181,
 12866,
 11500,
 10969,
 10829,
 10533,
 10345,
 9708,
 9382]

In [0]:
#
# Generate features - coupon discount and distance
#

def getDiscountType(row):
    if row == 'null':
        return 'null'
    elif ':' in row:
        return 1
    else:
        return 0

    
def convertRate(row):
    """Convert discount to rate"""
    if row == 'null':
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)

    
def getDiscountMan(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

    
def getDiscountJian(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

      
def goodDiscount(row):
  if row == 'null':
    return 0
  else:
    if row < 0.7:
      return 1
    else:
      return 0

def notgoodDiscount(row):
  if row == 'null':
    return 1
  else:
    if row > 0.7:
      return 1
    else:
      return 0    
    

def getDays(row):
  if row == 'null' or row == 'nan':
    return -1
  else:
    return float(row.replace('.0','')[-2:])


def isHotCoupon(row):
  if row == np.nan or row == 'null' or row == 'nan':
    return 0
  else:
    if row in top20_coupon:
        return 1
    else:
        return 0

  
  
def processData(df):
    
    # convert discunt_rate
    df['discount_rate'] = df['Discount_rate'].astype('str').apply(convertRate)
    df['discount_man'] = df['Discount_rate'].astype('str').apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].astype('str').apply(getDiscountJian)
    df['discount_type'] = df['Discount_rate'].astype('str').apply(getDiscountType)
    df['distance'] = df['Distance'].replace(np.nan, -1).astype(int)
    df['good_discount'] = df['discount_rate'].astype('float').apply(goodDiscount)
    df['notgood_discount'] = df['discount_rate'].astype('float').apply(notgoodDiscount)
    df['days_in_month'] = df['Date_received'].astype('str').apply(getDays)
    df['hot_coupon'] = df['Coupon_id'].apply(isHotCoupon)
    
    return df



In [23]:
sw.start()

dfoff = processData(dfoff)
dftest = processData(dftest)

sw.stop()
print('time elapsed:', sw.elapsed())

time elapsed: 00:00:09.790


In [24]:
print('dfoff[discount_rate]:',dfoff['discount_rate'].unique())
print('-')
print('dfoff[discount_rate]:',dftest['discount_rate'].unique())
print('-')
print('dfoff[distance]:', dfoff['distance'].unique())
print('-')
print('dftest[distance]:', dftest['distance'].unique())
print('-')
print('dftest[days_in_month]:', dftest['days_in_month'].unique())

dfoff[discount_rate]: [       nan 0.95       0.9        0.5        0.85       0.75
 0.83333333 0.8        0.93333333 0.7        0.6        0.86666667
 0.66666667 0.96666667 0.98       0.99       0.975      0.33333333
 0.2        0.4       ]
-
dfoff[discount_rate]: [0.86666667 0.95              nan 0.83333333 0.8        0.75
 0.66666667 0.9        0.96666667 0.6        0.7        0.5
 0.98       0.85       0.93333333 0.99       0.975      0.33333333]
-
dfoff[distance]: [ 0  1  2 -1 10  4  7  9  3  5  6  8]
-
dftest[distance]: [ 1  0 -1 10  7  2  9  4  8  3  6  5]
-
dftest[days_in_month]: [28. 13. -1. 16. 30. 19.  6. 18. 24. 23. 15. 21.  2.  5.  7.  1. 22. 11.
 10. 26. 20. 27. 25.  9. 14.  8.  3.  4. 12. 31. 17. 29.]


In [25]:
dfoff.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_rate,discount_man,discount_jian,discount_type,distance,good_discount,notgood_discount,days_in_month,hot_coupon
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,NaN,0,0,0,0,0,0,-1.0,0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0.95,20,1,1,0,0,1,17.0,0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0.95,20,1,1,0,0,1,19.0,0
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0.90,200,20,1,0,0,1,29.0,0
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0.90,200,20,1,1,0,1,29.0,0


In [26]:
print('Train Distance 類型：',dfoff['Distance'].unique())
print('Test Distance 類型：',dftest['Distance'].unique())

Train Distance 類型： [ 0.  1.  2. nan 10.  4.  7.  9.  3.  5.  6.  8.]
Test Distance 類型： [ 1.  0. nan 10.  7.  2.  9.  4.  8.  3.  6.  5.]


In [27]:
#
# Generate features - weekday acquired coupon   取得是星期幾拿到優惠券
#

def getWeekday(row):
    '''取得星期幾'''
    if (np.isnan(row)) or (row==-1):
        return row
    else:
        return pd.to_datetime(row, format = "%Y%m%d").dayofweek+1    # add one to make it from 0~6 -> 1~7

    
sw.start()

dfoff['weekday'] = dfoff['Date_received'].apply(getWeekday)
dftest['weekday'] = dftest['Date_received'].apply(getWeekday)

# weekday_type (weekend = 1) 1表示周末(六、日) 0為上班日(一～五)
dfoff['weekday_type'] = dfoff['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 ) # apply to train-set
dftest['weekday_type'] = dftest['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 )   # apply to test-set

sw.stop()
print('time elapsed:', sw.elapsed())

'''產生獨熱編碼'''
# 產生星期的欄位名稱
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
print(weekdaycols)

tmpdf = pd.get_dummies(dfoff['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dfoff[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(dftest['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf

time elapsed: 00:03:26.196
['weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [28]:
dfoff.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_rate,discount_man,discount_jian,discount_type,distance,good_discount,notgood_discount,days_in_month,hot_coupon,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,NaN,0,0,0,0,0,0,-1.0,0,NaN,0,0,0,0,0,0,0,0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0.95,20,1,1,0,0,1,17.0,0,3.0,0,0,0,1,0,0,0,0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0.95,20,1,1,0,0,1,19.0,0,6.0,0,0,0,0,0,0,1,0
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0.90,200,20,1,0,0,1,29.0,0,5.0,0,0,0,0,0,1,0,0
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0.90,200,20,1,1,0,1,29.0,0,5.0,0,0,0,0,0,1,0,0


In [0]:
#
# Creat target label
#
"""
According to the definition, 
1) buy with coupon within (include) 15 days ==> 1       # 在 n=15 天內有使用優惠券
2) buy with coupon but out of 15 days ==> 0             # 超過 n=15 天沒有使用優惠券
3) buy without coupon ==> -1 (we don't care)            # 都沒有使用優惠券，忽略這些資料 (普通消費紀錄)
"""

def labeling(row):
    if np.isnan(row['Date_received']):
        return -1       # 沒有領優惠券
    if not np.isnan(row['Date']):
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1   # 有在 15 天內使用優惠券
    return 0           # 沒有使用優惠券

  
dfoff["label"] = dfoff.apply(labeling, axis=1)

In [30]:
print(dfoff['label'].value_counts())

 0    710665
-1    413773
 1     36304
Name: label, dtype: int64


In [87]:
def split_train(row):
    sdd = pd.to_datetime("20160101", format="%Y%m%d")
    edd = pd.to_datetime("20160520", format="%Y%m%d")
    cdd = pd.to_datetime(row, format="%Y%m%d")
    is_train = True if cdd >= sdd and cdd <= edd else False
    return is_train
  
def split_valid(row):
    sdd = pd.to_datetime("20160417", format="%Y%m%d")
    edd = pd.to_datetime("20160531", format="%Y%m%d")
    cdd = pd.to_datetime(row, format="%Y%m%d")
    is_valid = True if cdd >= sdd and cdd <= edd else False
    return is_valid  


sw.start()

df = dfoff[dfoff['label'] != -1].copy()

#df["is_train"] = df["Date_received"].apply(split_train)
#df["is_valid"] = df["Date_received"].apply(split_valid)

df["is_train"] = df["Date"].apply(split_train)
df["is_valid"] = df["Date"].apply(split_valid)

train = df[df["is_train"]]
valid = df[df["is_valid"]]

train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)

sw.stop()
print('time elapsed:', sw.elapsed())

print("Train size: {}, #positive: {}".format(len(train), train["label"].sum()))
print("Valid size: {}, #positive: {}".format(len(valid), valid["label"].sum()))

time elapsed: 00:14:47.660
Train size: 42719, #positive: 36304
Valid size: 6009, #positive: 4470


In [88]:
# 建立特性欄位
original_feature = ['discount_rate',
                    'discount_type',
                    'discount_man', 
                    'discount_jian',
                    'distance', 
                    'weekday', 
                    'weekday_type',
                    'good_discount',
                    'notgood_discount',
                    'days_in_month',
                    'hot_coupon'
                    ] + weekdaycols

print(len(original_feature),original_feature)

18 ['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 'distance', 'weekday', 'weekday_type', 'good_discount', 'notgood_discount', 'days_in_month', 'hot_coupon', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [0]:
def check_model(data, predictors):

    classifier = lambda: SGDClassifier(
       loss='log',              # loss function: logistic regression
       penalty='elasticnet',    # L1 & L2
       fit_intercept=True,      # 是否存在截距，默認存在
       #max_iter=200, 
       shuffle=True,            # Whether or not the training data should be shuffled after each epoch
       n_jobs=1,                # The number of processors to use
       class_weight=None       # Weights associated with classes. If not given, all classes are supposed to have weight one.
    )
    
    
    # 管道機制使得參數集在新數據集（比如測試集）上的重複使用，管道機制實現了對全部步驟的流式化封裝和管理。
    model = Pipeline(steps=[
       ('ss', StandardScaler()), # transformer
       ('en', classifier())      # estimator
    ])

    parameters = {
       'en__alpha': [ 0.001, 0.01, 0.1],
       'en__l1_ratio': [ 0.001, 0.01, 0.1],
       'en__max_iter': [150, 200, 300, 500, 1000],
    }

    # StratifiedKFold用法類似Kfold，但是他是分層採樣，確保訓練集，測試集中各類別樣本的比例與原始數據集中相同。
    folder = StratifiedKFold(n_splits=3, random_state=0, shuffle=True)
    #folder = KFold(n_splits=4, random_state=0 ,shuffle=True)
    
    # Exhaustive search over specified parameter values for an estimator.
    grid_search = GridSearchCV(
       model, 
       parameters, 
       cv=folder, 
       n_jobs=-1,  # -1 means using all processors
       verbose=1)
    
    grid_search = grid_search.fit(data[predictors], 
                                 data['label'])

    return grid_search

In [90]:
sw.start()

predictors = original_feature
model = check_model(train, predictors)

sw.stop()
print('time elapsed:', sw.elapsed())

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    8.0s


time elapsed: 00:00:18.189


[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:   18.0s finished


In [91]:
# valid predict
y_valid_pred = model.predict_proba(valid[predictors])
valid1 = valid.copy()
valid1['pred_prob'] = y_valid_pred[:, 1]

valid1.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_rate,discount_man,discount_jian,discount_type,distance,good_discount,notgood_discount,days_in_month,hot_coupon,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,label,is_train,is_valid,pred_prob
0,6986436,6021,4448.0,20:5,1.0,20160414.0,20160421.0,0.750000,20,5,1,1,0,1,14.0,0,4.0,0,0,0,0,1,0,0,0,1,True,True,0.865909
1,6759108,582,3193.0,100:20,9.0,20160416.0,20160423.0,0.800000,100,20,1,9,0,1,16.0,0,6.0,0,0,0,0,0,0,1,0,1,True,True,0.770684
2,6759108,582,3193.0,100:20,9.0,20160423.0,20160502.0,0.800000,100,20,1,9,0,1,23.0,0,6.0,0,0,0,0,0,0,1,0,1,True,True,0.770684
3,2562404,7555,9871.0,30:5,10.0,20160418.0,20160420.0,0.833333,30,5,1,10,0,1,18.0,0,1.0,0,1,0,0,0,0,0,0,1,True,True,0.858484
4,1035609,7422,4727.0,0.8,2.0,20160428.0,20160428.0,0.800000,0,0,0,2,0,1,28.0,0,4.0,0,0,0,0,1,0,0,0,1,True,True,0.923761


In [92]:
# avgAUC calculation
vg = valid1.groupby(['Coupon_id'])
aucs = []
for i in vg:
    tmpdf = i[1] 
    if len(tmpdf['label'].unique()) != 2:
       continue
    fpr, tpr, thresholds = roc_curve(tmpdf['label'], tmpdf['pred_prob'], pos_label=1)
    aucs.append(auc(fpr, tpr))

print(np.average(aucs))

0.5104888060889163


In [93]:
y_valid_pred = model.predict_proba(valid[predictors].astype('float'))
valid1 = valid.copy()
valid1['pred_prob'] = y_valid_pred[:, 1]


auc_score = roc_auc_score(y_true=valid.label, y_score=y_valid_pred[:,1])
acc = accuracy_score(y_true=valid.label, y_pred=y_valid_pred.argmax(axis=1))

print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

Validation AUC: 0.645, Accuracy: 0.744


In [94]:
targetset = dftest.copy()
print(targetset.shape)

targetset = targetset[~targetset.Coupon_id.isna()]
targetset.reset_index(drop=True, inplace=True)
testset = targetset[predictors].copy()

y_test_pred = model.predict_proba(testset[predictors].astype('float'))
test1 = testset.copy()

test1['pred_prob'] = y_test_pred[:, 1]
print(test1.shape)

(594142, 24)
(306313, 19)


In [95]:
output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], test1["pred_prob"]), axis=1)
print(output.shape)

(306313, 4)


In [0]:
output.loc[:, "User_id"] = output["User_id"].apply(lambda x:str(int(x)))
output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x:str(int(x)))
output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x:str(int(x)))
output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
output.reset_index(drop=True, inplace=True)

In [97]:
### NOTE: YOUR SUBMITION FILE SHOULD HAVE COLUMN NAME: uid, label
out = output.groupby("uid", as_index=False).mean()
out = out[["uid", "pred_prob"]]
out.columns = ["uid", "label"]

out.head()

,uid,label
0,1000020_2705_20160519,0.866680
1,1000020_8192_20160513,0.865027
2,1000065_1455_20160527,0.866727
3,1000085_8067_20160513,0.859656
4,1000086_2418_20160613,0.866383


In [98]:
print(out.shape)

(304096, 2)


In [0]:
# 將結果存為 CSV 檔
out.to_csv("midterm_exam_23.csv", header=["uid", "label"], index=False) # submission format

In [100]:
!zip midterm_exam_23.zip midterm_exam_23.csv

  adding: midterm_exam_23.csv (deflated 77%)
